In [ ]:
from modelarts.session import Session
session = Session()

session.download_data(
    bucket_path="modelarts-labs/notebook/DL_image_recognition/image_recognition.tar.gz",
    path="./image_recognition.tar.gz")

# 使用tar命令解压资源包
!tar xf ./image_recognition.tar.gz

# 清理压缩包
!rm -f ./image_recognition.tar.gz

In [ ]:
session.download_data(
    bucket_path="ai-course-ansel/efficientnet-b0.tar.gz",
    path="./efficientnet-b0.tar.gz")

# 使用tar命令解压资源包
!tar xf ./efficientnet-b0.tar.gz

# 清理压缩包
!rm -f ./efficientnet-b0.tar.gz

In [1]:
!ls

data					lost+found
dog_and_cat_efficientnetb0-Copy1.ipynb	main.py
efficientnet-b0				preprocessing.py
efficientnet_builder.py			__pycache__
efficientnet_model.py			README.md
eval_ckpt_example.ipynb			upgrade tensorflow 1.14.0.ipynb
eval_ckpt_main.py			utils.py
imagenet_input.py


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
#tf.enable_eager_execution()
tf.__version__

'1.14.0'

In [3]:
import os
import random
from sklearn.model_selection import train_test_split
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [4]:
DATA_DIR = './data/'
model_name = model_name = 'efficientnet-b0'
ckpt_dir = './efficientnet-b0/'
export_ckpt_dir = None
BATCH_SIZE = 32
#MEAN_RGB = [0.485 * 255, 0.456 * 255, 0.406 * 255]
#STDDEV_RGB = [0.229 * 255, 0.224 * 255, 0.225 * 255]

In [5]:
all_image_paths = [DATA_DIR+i for i in os.listdir(DATA_DIR)]
random.shuffle(all_image_paths)

In [6]:
all_image_labels=[]
for i in all_image_paths:
    if 'dog' in i:
        all_image_labels.append(1)
    else:
        all_image_labels.append(0)

In [7]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    #image -= tf.constant(MEAN_RGB, shape=[1, 1, 3], dtype=image.dtype)
    #image /= tf.constant(STDDEV_RGB, shape=[1, 1, 3], dtype=image.dtype)
    return image

In [8]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

In [9]:
train_paths, test_paths, train_labels, test_labels = train_test_split(all_image_paths, all_image_labels, test_size=0.2, random_state=10)
tra_paths,val_paths,tra_labels,val_labels = train_test_split(train_paths,train_labels,test_size=0.2,random_state=10)

In [10]:
def build_dataset(images,labels):
    ds = tf.data.Dataset.from_tensor_slices((tra_paths, tra_labels))
    # The tuples are unpacked into the positional arguments of the mapped function
    def load_and_preprocess_from_path_label(path, label):
        return load_and_preprocess_image(path), label
    image_label_ds = ds.map(load_and_preprocess_from_path_label)
    ds = image_label_ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=len(labels)))
    ds = ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
    return ds

In [11]:
train_batches = build_dataset(tra_paths,tra_labels)
test_batches = build_dataset(test_paths,test_labels)
val_batches = build_dataset(val_paths,val_labels)

W0711 13:13:23.924025 140484842403584 deprecation.py:323] From <ipython-input-10-07c1cf0db198>:7: shuffle_and_repeat (from tensorflow.python.data.experimental.ops.shuffle_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.


In [ ]:
#image_batch, label_batch = next(iter(ds))

In [12]:
import efficientnet_model
import efficientnet_builder

W0711 13:13:30.855360 140484842403584 deprecation_wrapper.py:119] From /home/jovyan/work/utils.py:86: The name tf.layers.BatchNormalization is deprecated. Please use tf.compat.v1.layers.BatchNormalization instead.

W0711 13:13:30.856772 140484842403584 deprecation_wrapper.py:119] From /home/jovyan/work/utils.py:200: The name tf.layers.Layer is deprecated. Please use tf.compat.v1.layers.Layer instead.



In [13]:
blocks_args, global_params = efficientnet_builder.get_model_params(model_name, None)
input = tf.keras.Input(shape=(224,224,3))
basemodel = efficientnet_model.Model(blocks_args, global_params)
with tf.variable_scope(model_name):
    features = basemodel(input, training=True, features_only=True)
    tf.train.init_from_checkpoint('./efficientnet-b0/model.ckpt', {'efficientnet-b0/':'efficientnet-b0/'})

W0711 13:13:31.078946 140484842403584 deprecation_wrapper.py:119] From /home/jovyan/work/efficientnet_builder.py:157: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

W0711 13:13:31.088542 140484842403584 deprecation.py:506] From /opt/conda/envs/python36_tf/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0711 13:13:31.090062 140484842403584 deprecation_wrapper.py:119] From /home/jovyan/work/efficientnet_model.py:201: The name tf.layers.Conv2D is deprecated. Please use tf.compat.v1.layers.Conv2D instead.

W0711 13:13:31.119100 140484842403584 deprecation_wrapper.py:119] From /home/jovyan/work/efficientnet_model.py:372: The name tf.layers.Dense is deprecated. Please use tf.compat

In [14]:
base_model = tf.keras.Model(inputs=input,outputs=features)
base_model.trainable = False
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [15]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

W0711 13:13:41.273414 140484842403584 deprecation.py:323] From /opt/conda/envs/python36_tf/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [16]:
initial_epochs = 10
steps_per_epoch = round(len(tra_labels))//BATCH_SIZE
validation_steps = round(len(val_labels))//BATCH_SIZE
print(steps_per_epoch)

500


In [17]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    steps_per_epoch = steps_per_epoch,
                    validation_data = val_batches,
                    validation_steps = validation_steps )

Epoch 1/10
500/500 [==============================] - 637s 1s/step - loss: 0.2118 - acc: 0.9094 - val_loss: 0.1232 - val_acc: 0.9542
Epoch 2/10
500/500 [==============================] - 606s 1s/step - loss: 0.1368 - acc: 0.9427 - val_loss: 0.1074 - val_acc: 0.9563
Epoch 3/10
500/500 [==============================] - 610s 1s/step - loss: 0.1234 - acc: 0.9509 - val_loss: 0.1092 - val_acc: 0.9553
Epoch 4/10
500/500 [==============================] - 610s 1s/step - loss: 0.1166 - acc: 0.9531 - val_loss: 0.0925 - val_acc: 0.9622
Epoch 5/10
500/500 [==============================] - 611s 1s/step - loss: 0.1110 - acc: 0.9554 - val_loss: 0.0963 - val_acc: 0.9617
Epoch 6/10
500/500 [==============================] - 610s 1s/step - loss: 0.0999 - acc: 0.9606 - val_loss: 0.0807 - val_acc: 0.9710
Epoch 7/10
500/500 [==============================] - 610s 1s/step - loss: 0.1010 - acc: 0.9592 - val_loss: 0.0709 - val_acc: 0.9703
Epoch 8/10
500/500 [==============================] - 610s 1s/step - 

In [18]:
OUTPUT = 'output'

if not os.path.exists(OUTPUT):
    os.mkdir(OUTPUT)

model.save_weights(os.path.join(OUTPUT, 'b0.h5'))

In [19]:
steps = round(len(test_labels))//BATCH_SIZE

In [20]:
model.evaluate(test_batches,steps=steps)

156/156 [==============================] - 160s 1s/step - loss: 0.0741 - acc: 0.9726


[0.07409767107142565, 0.9725561]